In [141]:
# Загрузите данные из файла data-logistic.csv. 
# Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.
import pandas as pd
data = pd.read_csv('data\\data-logistic.csv', header=None)
y = data[0]
X = data.drop(0, 1)

In [142]:
y[:5]

0   -1
1    1
2   -1
3    1
4    1
Name: 0, dtype: int64

In [143]:
X[:5]

,1,2
0,-0.663827,-0.138526
1,1.994596,2.468025
2,-1.247395,0.749425
3,2.309374,1.899836
4,0.849143,2.407750


In [144]:
# Убедитесь, что выше выписаны правильные формулы для градиентного спуска. 
import math
def update_w(X, y, w, k, C):
    l = len(X)
    summa_w1 = 0
    summa_w2 = 0
    for i in range(0, l):
        summa_w1 += y[i] * X[1][i] * (1.0 - 1.0 / (1.0 + math.exp(-y[i] * (w[0]*X[1][i] + w[1]*X[2][i]))))
        summa_w2 += y[i] * X[2][i] * (1.0 - 1.0 / (1.0 + math.exp(-y[i] * (w[0]*X[1][i] + w[1]*X[2][i]))))

    new_w1 = w[0] + (k * (1.0 / l) * summa_w1) - k * C * w[0]
    new_w2 = w[1] + (k * (1.0 / l) * summa_w2) - k * C * w[1]
    return [new_w1, new_w2]

In [145]:
# Проверка обновления весов
print(update_w(X, y, [0.0, 0.0], 0.1, 10.0))

[0.035731266398741086, 0.032459971278388684]


In [146]:
# Проверка Пифагора и Евклида 
from scipy.spatial import distance
a = [0, 0]
b = [3, 4]
print(distance.euclidean(a,b))

5.0


In [147]:
# Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. 
# Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

# Запустите градиентный спуск и доведите до сходимости 
# (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). 
# Рекомендуется ограничить сверху число итераций десятью тысячами.

def gradient_descent(X, y, C = 0.0, w0 = [0.0, 0.0], k = 0.1, error = 1e-5, max_steps = 10000):
    l = len(X) # длина выборки
    w = w0
    
    for step in range(max_steps):
        new_w = update_w(X, y, w, k, C)
        
        dist = distance.euclidean(new_w, w)
        if dist < error:
            print('Достигли заданной погрешности. Шаг ', step)
            print(new_w)
            break
        else:
            w = new_w

    return w   

In [148]:
w1 = gradient_descent(X, y)

Достигли заданной погрешности. Шаг  243
[0.28781162047177639, 0.09198330215925439]


In [149]:
w2 = gradient_descent(X, y, 10.0)

Достигли заданной погрешности. Шаг  7
[0.028558754546234223, 0.024780137249735559]


In [150]:
# Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? 

# Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. 
# Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)).

from sklearn.metrics import roc_auc_score

def a(X, w):
    return 1.0 / (1.0 + math.exp(-w[0] * X[1] - w[1] * X[2]))

est1 = X.apply(lambda x: a(x, w1), axis=1)
est2 = X.apply(lambda x: a(x, w2), axis=1)

auc1 = roc_auc_score(y, est1)
auc2 = roc_auc_score(y, est2)

In [151]:
auc1, auc2
# 0.927 0.936

(0.92685714285714282, 0.93628571428571417)